![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FVertex+AI+Search&dt=Vertex+AI+Search+Document+Q%26A+Using+Extractive+Segments+-+MLB+Rules+For+Baseball.ipynb)

# UmpireBot - MLB Rules For Baseball

Based on [Vertex AI Search](https://cloud.google.com/enterprise-search) (FKA) Generative AI App Builder Enterprise Search

**What?**

Ask questions of the rules for MLB and get answers with specific references to official rules.

**Sources:**

The official MLB rules are at the links below:
- [MLB 2022 Rule Book](https://img.mlbstatic.com/mlb-images/image/upload/mlb/hhvryxqioipb87os1puw.pdf)
- [MLB 2023 Rule Book](https://img.mlbstatic.com/mlb-images/image/upload/mlb/wqn5ah4c3qtivwx3jatm.pdf)

**Tools:**
- Vertex AI Search
  - This [link](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#cloud-storage) will provide the steps to create a datastore using unstructured data(pdf is used in this example) which will be stored in Cloud Storage.
- Vertex AI GenAI Language Model API
  - `vertexai.language_models.TextGenerationModel.from_pretrained('text-bison@latest')`

---

**Google Cloud Vertex AI Generative AI Support**

Vertex AI Generative AI gives access to Google's large genearative AI models and also enables you to test, tune, and deploy them for your applications. Get an overview [here](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

---

**How:**

- Vertex AI Search creates three components: Apps, Engines and Datastores
  - More information about App and Datastore can be found [here](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest)
- Ask a question
  - Query the datastore using the Python Client API
  - API will search the datastore for related information in the documents using vector similarity
  - Retrieve the content associated from Datastore
  - Prepare a prompt to answer the question using the retrieved search results as context
- Present the response as an answer with links to the related documents (sections of the document).

**What is Unique?**

An LLM is likely unexposed to a users private content.  This appoach constructs a summarization prompt for an LLM by first retriving context for the question from the users documents using embeddings.  This also allows the response to be accompanied by direct reference to the users documentation used in the prompt.

**Notes**

- An LLM is likely trained on many sources that probably include lots of general knowledge, even information like what is used here.  It is also likely knowledgable of past outdated information which can be a benefit - or detriment - to accuracy.  This approach directly uses the version of the document that currently applies.

**Prerequisites**

This notebook uses Vertex AI and Generative AI App Builder for processing data while also using Google Cloud Storage for data storage and retrieval.  If you are running this notebook from Colab or another environment where your user id is authenticated then your account will need roles/permissions that allow working with these services.  If you are running this from a Vertex AI Workbench Notebook instance then it is running as a service account which will need the roles/permission that allow working with these services.

This notebook requires an App and Data Store created with the PDF downloaded from MLB website mentioned above.

**Vertex AI Search Setup**

Vertex AI Search Setup can be found [here](./vertex_search_setup.md)

**References**

- [Vertex AI GenAI Studio](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview)
    - Vertex AI [Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest)
- Vertex AI Search [Python Client](https://cloud.google.com/generative-ai-app-builder/docs/preview-search-results?_ga=2.154471792.-952990647.1684293017)
   

### Installs

In [1]:
#For Accessing Vertex AI GenAI Studio API
!pip install google-cloud-aiplatform -q --user

#For Accessing Vertex AI Search API
!pip install google-cloud-discoveryengine -q --user

**Restart Runtime**

In [2]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Setup

Inputs

In [1]:
# project_id = "vertexai-demo-ltfpzhaw"
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'vertexai-demo-ltfpzhaw'

In [50]:
location = "global" # valid values are global,us,eu.
region = "us-central1"
search_engine_id = "mlb-rule-book_1695652295607" # update search_engine_id with your datastore id.
serving_config_id = "default_config" # default_config is the only valid value.
search_query = "What is a base?"
content_list = []
NUM_EXAMPLES_TOP = 1
MAX_SEGMENT_COUNT = 1

Packages

In [51]:
from typing import List
import json
from google.protobuf.json_format import MessageToDict
import google.cloud.discoveryengine_v1beta as de
import vertexai
import vertexai.language_models as lm
from google.cloud import aiplatform
from google.api_core.client_options import ClientOptions
import IPython

In [52]:
vertex_client = vertexai.init(project = PROJECT_ID, location = region)
aip_client = aiplatform.init(project = PROJECT_ID, location = region)

In [53]:
textgen_model = lm.TextGenerationModel.from_pretrained('text-bison@latest')

## Using Vertex AI Search with Extractive Segments

In [72]:
def search_with_extractive_segments(
    project_id: str,
    location: str,
    search_engine_id: str,
    serving_config_id: str,
    search_query: str,
) -> List[de.SearchResponse.SearchResult]:
    
    
    api_endpoint = (
    "discoveryengine.googleapis.com"
    if location == "global"
    else f"{location}-discoveryengine.googleapis.com"
    )


    client = de.SearchServiceClient(client_options=ClientOptions(api_endpoint=api_endpoint))
    
    content_search_spec = de.SearchRequest.ContentSearchSpec(snippet_spec=de.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True),
    summary_spec=de.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=NUM_EXAMPLES_TOP, include_citations=True),
    extractive_content_spec=de.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=NUM_EXAMPLES_TOP,
            max_extractive_segment_count=MAX_SEGMENT_COUNT))

    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=search_engine_id,
        serving_config=serving_config_id
    )

    request = de.SearchRequest(
        serving_config=serving_config, query=search_query, page_size=20, content_search_spec = content_search_spec)

    response = client.search(request)
    for result in response.results:
        data = MessageToDict(result.document._pb)
        derived_struct_data = data.get("derivedStructData")
        if derived_struct_data is not None:
            extractive_segments = derived_struct_data.get("extractive_segments")
            if extractive_segments is not None and len(extractive_segments) > 0:
                # print(extractive_segments)
                content = extractive_segments[0].get("content")
                page_number = extractive_segments[0].get("pageNumber")
                content_list.append(content)
    return content_list

In [75]:
IPython.display.Markdown(''.join(search_with_extractive_segments(PROJECT_ID,location,search_engine_id,serving_config_id,search_query)))

146

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.143

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.

## Using Text-Bison for generating response based on the search result from Vertex AI Search

### Text-Bison Response using Extractive Segments from Vertex AI Search

<u>**Note**</u>: **Sources** section in the response below contains the page number and the retrieved search result from Vertex AI Search Datastore.

In [78]:
def get_llm_response_with_extractive_segments(search_query):

    prompt_context = search_with_extractive_segments(PROJECT_ID,location,search_engine_id,serving_config_id,search_query)
    prompt = f"""
Give a detailed answer to the Question below using only this context:{prompt_context}.
Question: {search_query}
Answer and Explanation:
    """
    text_gen_response = textgen_model.predict(prompt = prompt, max_output_tokens = 1024, temperature= 0)
    llm_response = f"""## Response\n\n### Question\n\n{search_query}\n\n### Answer\n\n{text_gen_response}\n\n### Sources\n\n{''.join(prompt_context)}"""
    content_list.clear()
    return IPython.display.Markdown(llm_response)

In [79]:
get_llm_response_with_extractive_segments(search_query)

## Response

### Question

What is a base?

### Answer

 A base is one of four points which must be touched by a runner in order to score a run; more usually applied to the canvas bags and the rubber plate which mark the base points.

### Sources

146

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.143

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.

In [80]:
get_llm_response_with_extractive_segments("Is a rule broken if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball?")

## Response

### Question

Is a rule broken if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball?

### Answer

 Yes, it is a rule violation. 

According to  Rule 5.02(c)(ii), at the time the pitcher releases the ball for delivery to the batter, the defensive team must have a minimum of four players (in addition to the pitcher and the catcher) with both feet completely in front of the outer boundary of the infield dirt, at least two of which must be positioned with both feet entirely on each side of second base. 

Therefore, if three infielders are positioned on the same side of the field where the batter is more likely to hit the ball, it is a violation of this rule.

### Sources

19

Rule 5.02(c)
(ii) at the time the pitcher releases the ball for delivery to
the batter, the defensive team must have a minimum
of four players (in addition to the pitcher and the
catcher) with both feet completely in front of the outer
boundary of the infield dirt, at least two of which must
be positioned with both feet entirely on each side of
second base; and
(iii) from the time the pitcher releases the ball to deliver
the first pitch to the first batter of a half inning, the two
infielders on each side of second base may not switch
sides or move to a position other than their side of the
infield for the entirety of that inning. Notwithstanding
the foregoing, any infielder may switch sides, or move
to any other position at the time of a substitution for one
of the defensive players (other than a pitching change
that substitutes the pitcher for a player not already in
the game). Any player who legally replaces an infielder
during an inning also may not switch sides or move to
a position other than their side of the infield from the
time the pitcher releases the ball to deliver the first pitch
following the substitution to the end of that half inning
(except upon the occurrence of a subsequent substitution
during that half inning).
Rule 5.02(c) Comment. Umpires should bear in mind that the
purpose of the Infielder Positioning rule is to prevent the defense
from having more than two infielders on either side of second
base in an effort to anticipate where the batter will hit the ball
prior to delivery of the pitch. If, in the judgment of the umpire,
any fielder attempts to circumvent the purposes of this Rule
5.02(c), the umpire shall assess the penalty described below.

PENALTY: If the defensive team violates Rule 5.02(c), the
pitch shall be called a “ball” and the ball is dead, unless the
batter reaches first base on a hit, an error, a base on balls, a
hit batter, or otherwise, and all other runners advance at least
one base, in which case the play proceeds without reference
to the violation. If any other play follows the violation (e.g.,
sacrifice fly, sacrifice bunt, etc.), the manager of the offense30

Rule 5.06(b)(4)
ball hit the ground, passes a receiving fielder or goes out of play
into the stands.
The position of the batter-runner at the time the wild throw left
the thrower’s hand is the key in deciding the award of bases.
If the batter-runner has not reached first base, the award is
two bases at the time the pitch was made for all runners. The
decision as to whether the batter-runner has reached first base
before the throw is a judgment call.
If an unusual play arises where a first throw by an infielder goes
into stands or dugout but the batter did not become a runner
(such as catcher throwing ball into stands in attempt to get
runner from third trying to score on passed ball or wild pitch)
award of two bases shall be from the position of the runners at
the time of the throw. (For the purpose of Rule 5.06(b)(4)(G) a
catcher is considered an infielder.)
PLAY—Runner on first base, batter hits a ball to the shortstop,
who throws to second base too late to get runner at second,
and second baseman throws toward first base after batter has
crossed first base.
RULING: Runner at second scores. (On this play, only if batter
runner is past first base when throw is made is he awarded third
base.)

(H)  One base, if a ball, pitched to the batter, or thrown by
the pitcher from his position on the pitcher’s plate to
a base to catch a runner, goes into a stand or a bench,
or over or through a field fence or backstop. The ball
is dead;
APPROVED RULING: When a wild pitch or passed ball goes
through or by the catcher, or deflects off the catcher, and goes
directly into the dugout, stands, above the break, or any area
where the ball is dead, the awarding of bases shall be one base.
One base shall also be awarded if the pitcher while in contact
with the rubber, throws to a base, and the throw goes directly
into the stands or into any area where the ball is dead.
If, however, the pitched or thrown ball goes through or by
the catcher or through the fielder, and remains on the playing

In [81]:
get_llm_response_with_extractive_segments("What is the definition of a balk?")

## Response

### Question

What is the definition of a balk?

### Answer

 A balk is an illegal act by the pitcher with a runner or runners on base, entitling all runners to advance one base. 

### Sources

146

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.143

Definitions of Terms

DEFINITIONS OF TERMS
(All definitions are listed alphabetically)
ADJUDGED is a judgment decision by the umpire.
An APPEAL is the act of a fielder in claiming violation of the rules
by the offensive team.
A BALK is an illegal act by the pitcher with a runner or runners on
base, entitling all runners to advance one base.
A BALL is a pitch which does not enter the strike zone in flight and
is not struck at by the batter. If the pitch touches the ground and bounces
through the strike zone it is a “ball.”
A BASE is one of four points which must be touched by a runner
in order to score a run; more usually applied to the canvas bags and the
rubber plate which mark the base points.
A BASE COACH is a team member in uniform who is stationed in
the coach’s box at first or third base to direct the batter and the runners.
A BASE ON BALLS is an award of first base granted to a batter
who, during his time at bat, receives four pitches outside the strike zone
or following a signal from the defensive team’s manager to the umpire
that he intends to intentionally walk the batter. If the manager informs the
umpire of this intention, the umpire shall award the batter first base as if
the batter had received four pitches outside the strike zone.
A BATTER is an offensive player who takes his position in the
batter’s box.
BATTER-RUNNER is a term that identifies the offensive player
who has just finished his time at bat until he is put out or until the play on
which he became a runner ends.
The BATTER’S BOX is the area within which the batter shall stand
during his time at bat.
The BATTERY is the pitcher and catcher.
BENCH OR DUGOUT is the seating facilities reserved for
players, substitutes and other team members in uniform when they are not
actively engaged on the playing field.

In [45]:
get_llm_response_with_extractive_segments("A batter hits a ball that goes over the fence, but it is caught by a fan in the stands. Is the ball a home run?")

## Response

### Question

A batter hits a ball that goes over the fence, but it is caught by a fan in the stands. Is the ball a home run?

### Answer

 No, the ball is not a home run. 
     According to the rule 5.05(a)(5), a fair ball passes over a fence or into the stands at a distance from home base of 250 feet or more. Such hit entitles the batter to a home run when he shall have touched all bases legally. 
     In this case, the ball is caught by a fan in the stands, which means it did not pass over the fence. Therefore, it is not a home run.

### Sources

24

Rule 5.04(c) to 5.05(a)(6)

(c) Completing Time at Bat
A batter has legally completed his time at bat when he is put out
or becomes a runner.

5.05 When the Batter Becomes a Runner
(a) The batter becomes a runner when:
(1)  He hits a fair ball;
Rule 5.05(a) Comment: If the batter hits a pitch that touches the
ground first, the ensuing action shall be the same as if he hit the
ball in flight.
(2)  The third strike called by the umpire is not caught,
providing (1) first base is unoccupied, or (2) first base is
occupied with two out;
Rule 5.05(a)(2) Comment: A batter who does not realize his
situation on a third strike not caught, and who is not in the
process of running to first base, shall be declared out once he
leaves the dirt circle surrounding home plate.
(3)  If the pitch touches the ground and bounces through the
strike zone it is a “ball.” If such a pitch touches the batter,
he shall be awarded first base. If the batter swings at such
a pitch after two strikes, the ball cannot be caught, for the
purposes of Rule 5.05(b) and 5.09(a)(3).
(4)  A fair ball, after having passed a fielder other than the
pitcher, or after having been touched by a fielder, including
the pitcher, shall touch an umpire or runner on fair territory;
(5)  A fair ball passes over a fence or into the stands at a distance
from home base of 250 feet or more. Such hit entitles the
batter to a home run when he shall have touched all bases
legally. A fair fly ball that passes out of the playing field
at a point less than 250 feet from home base shall entitle
the batter to advance to second base only;
(6)  A fair ball, after touching the ground, bounds into the
stands, or passes through, over or under a fence, or through23

Rule 5.05(a)(2) to 5.05(a)(8)

(2)  The third strike called by the umpire is not caught,
providing (1) first base is unoccupied, or (2) first base is
occupied with two out;
Rule 5.05(a)(2) Comment: A batter who does not realize his
situation on a third strike not caught, and who is not in the
process of running to first base, shall be declared out once he
leaves the dirt circle surrounding home plate.
(3)  If the pitch touches the ground and bounces through the
strike zone it is a “ball.” If such a pitch touches the batter,
he shall be awarded first base. If the batter swings at such
a pitch after two strikes, the ball cannot be caught, for the
purposes of Rule 5.05(b) and 5.09(a)(3).
(4)  A fair ball, after having passed a fielder other than
the pitcher, or after having been touched by a fielder,
including the pitcher, shall touch an umpire or runner on
fair territory;
(5)  A fair ball passes over a fence or into the stands at a distance
from home base of 250 feet or more. Such hit entitles the
batter to a home run when he shall have touched all bases
legally. A fair fly ball that passes out of the playing field
at a point less than 250 feet from home base shall entitle
the batter to advance to second base only;
(6)  A fair ball, after touching the ground, bounds into the
stands, or passes through, over or under a fence, or through
or under a scoreboard, or through or under shrubbery, or
vines on the fence, in which case the batter and the runners
shall be entitled to advance two bases;
(7)  Any fair ball which, either before or after touching the
ground, passes through or under a fence, or through or
under a scoreboard, or through any opening in the fence or
scoreboard, or through or under shrubbery, or vines on the
fence, or which sticks in a fence or scoreboard, in which
case the batter and the runners shall be entitled to two bases;
(8)  Any bounding fair ball is deflected by the fielder into the
stands, or over or under a fence on fair or foul territory, in

In [83]:
get_llm_response_with_extractive_segments("What is the dimension of base?")

## Response

### Question

What is the dimension of base?

### Answer

 The dimension of the base is 17 inches square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8½ inches and the remaining two sides are 12 inches and set at an angle to make a point.

### Sources

3

Rule 2.01 to 2.03

The foul lines and all other playing lines indicated in the diagrams
by solid black lines shall be marked with paint or non-toxic and
non-burning chalk or other white material.
The grass lines and dimensions shown on the diagrams are those
used in many fields, but they are not mandatory and each Club shall
determine the size and shape of the grassed and bare areas of its
playing field.
NOTE: (a) Any Playing Field constructed by a professional
Club after June 1, 1958, shall provide a minimum distance of
325 feet from home base to the nearest fence, stand or other
obstruction on the right and left field foul lines, and a minimum
distance of 400 feet to the center field fence.
(b) No existing playing field shall be remodeled after
June 1, 1958, in such manner as to reduce the distance from
home base to the foul poles and to the center field fence below
the minimum specified in paragraph (a) above.
2.02 Home Base
Home base shall be marked by a five-sided slab of whitened rubber.
It shall be a 17-inch square with two of the corners removed so that
one edge is 17 inches long, two adjacent sides are 8½ inches and
the remaining two sides are 12 inches and set at an angle to make a
point. It shall be set in the ground with the point at the intersection
of the lines extending from home base to first base and to third base;
with the 17-inch edge facing the pitcher’s plate, and the two 12-inch
edges coinciding with the first and third base lines. The top edges of
home base shall be beveled and the base shall be fixed in the ground
level with the ground surface. (See drawing D in Appendix 2.)
2.03 The Bases
First, second and third bases shall be marked by white canvas or
rubber-covered bags, securely attached to the ground as indicated in
Diagram 2. The first and third base bags shall be entirely within the
infield. The second base bag shall be centered on second base. The
bags shall be 15 inches square, not less than three nor more than five
inches thick, and filled with soft material.3

Rule 2.01 to 2.02

The foul lines and all other playing lines indicated in the diagrams
by solid black lines shall be marked with paint or non-toxic and
non-burning chalk or other white material.
The grass lines and dimensions shown on the diagrams are those
used in many fields, but they are not mandatory and each Club
shall determine the size and shape of the grassed and bare areas
of its playing field; provided, however, that the grass line of the
outer boundary of the infield dirt shall be a 95-foot radius from the
center of the pitcher’s plate. A Club will be considered to be in
compliance with this provision if the average distance of the grass
line of the outer boundary of the infield dirt is less than 96 feet but
more than 94 feet, and no individual measurement is more than 96
feet or less than 94 feet.
NOTE: (a) Any Playing Field constructed by a professional
Club after June 1, 1958, shall provide a minimum distance of
325 feet from home base to the nearest fence, stand or other
obstruction on the right and left field foul lines, and a minimum
distance of 400 feet to the center field fence.
(b) No existing playing field shall be remodeled after
June 1, 1958, in such manner as to reduce the distance from
home base to the foul poles and to the center field fence below
the minimum specified in paragraph (a) above.

2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber.
It shall be a 17-inch square with two of the corners removed so that
one edge is 17 inches long, two adjacent sides are 8½ inches and
the remaining two sides are 12 inches and set at an angle to make a
point. It shall be set in the ground with the point at the intersection
of the lines extending from home base to first base and to third base;
with the 17-inch edge facing the pitcher’s plate, and the two 12-inch
edges coinciding with the first and third base lines. The top edges of
home base shall be beveled and the base shall be fixed in the ground
level with the ground surface. (See drawing D in Appendix 2.)